## Exercise 1: Basic Custom Training Loop

**Setup Environment**

In [1]:
import os
import warnings
import numpy as np

import tensorflow as tf 
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input

# Suppress all Python warnings
warnings.filterwarnings('ignore')

# Set TensorFlow log level to suppress warnings and info messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Setup the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() 
x_train, x_test = x_train / 255.0, x_test / 255.0 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)

**Define Model**

In [2]:
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10)
])

**Define Loss Function and Optimizer**

In [3]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
optimizer = tf.keras.optimizers.Adam()

**Implement Custom Training Loop**

In [4]:
epochs = 2

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)
for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)  # Forward pass
            loss_value = loss_fn(y_batch_train, logits)  # Compute loss

        # Compute gradients and update weights
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Logging the loss every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()}')

Start of epoch 1
Epoch 1 Step 0: Loss = 2.435736656188965
Epoch 1 Step 200: Loss = 0.35337692499160767
Epoch 1 Step 400: Loss = 0.199442058801651
Epoch 1 Step 600: Loss = 0.1928015649318695
Epoch 1 Step 800: Loss = 0.16863012313842773
Epoch 1 Step 1000: Loss = 0.45767319202423096
Epoch 1 Step 1200: Loss = 0.16339987516403198
Epoch 1 Step 1400: Loss = 0.2781286835670471
Epoch 1 Step 1600: Loss = 0.2289547473192215
Epoch 1 Step 1800: Loss = 0.1782616823911667
Start of epoch 2
Epoch 2 Step 0: Loss = 0.07691352069377899
Epoch 2 Step 200: Loss = 0.15640394389629364
Epoch 2 Step 400: Loss = 0.10423268377780914
Epoch 2 Step 600: Loss = 0.05262263864278793
Epoch 2 Step 800: Loss = 0.08984110504388809
Epoch 2 Step 1000: Loss = 0.2653706967830658
Epoch 2 Step 1200: Loss = 0.07123812288045883
Epoch 2 Step 1400: Loss = 0.19031307101249695
Epoch 2 Step 1600: Loss = 0.15378209948539734
Epoch 2 Step 1800: Loss = 0.09773679822683334


## Exercise 2: Adding Accuracy Metric

**Setup Environment**

In [5]:
# Setup the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize the pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0 

# Create a batched dataset for training
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)

**Define Model**

In [6]:
model = Sequential([ 
    Flatten(input_shape=(28, 28)),  # Flatten the input to a 1D vector
    Dense(128, activation='relu'),  # First hidden layer with 128 neurons and ReLU activation
    Dense(10)  # Output layer with 10 neurons for the 10 classes (digits 0-9)
])

**Define Loss Function, Optimizer, and Metric**

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Loss function for multi-class classification
optimizer = tf.keras.optimizers.Adam()  # Adam optimizer for efficient training
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()  # Metric to track accuracy during training

**Implement Custom Training Loop with Accuracy**

In [8]:
epochs = 5  # Number of epochs for training

for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')
    
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # Forward pass: Compute predictions
            logits = model(x_batch_train, training=True)
            # Compute loss
            loss_value = loss_fn(y_batch_train, logits)
        
        # Compute gradients
        grads = tape.gradient(loss_value, model.trainable_weights)
        # Apply gradients to update model weights
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        # Update the accuracy metric
        accuracy_metric.update_state(y_batch_train, logits)

        # Log the loss and accuracy every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()} Accuracy = {accuracy_metric.result().numpy()}')
    
    # Reset the metric at the end of each epoch
    accuracy_metric.reset_state()

Start of epoch 1
Epoch 1 Step 0: Loss = 2.302577495574951 Accuracy = 0.15625
Epoch 1 Step 200: Loss = 0.34634149074554443 Accuracy = 0.8339552283287048
Epoch 1 Step 400: Loss = 0.19706083834171295 Accuracy = 0.8673628568649292
Epoch 1 Step 600: Loss = 0.19793710112571716 Accuracy = 0.8826954960823059
Epoch 1 Step 800: Loss = 0.16251564025878906 Accuracy = 0.8955602645874023
Epoch 1 Step 1000: Loss = 0.3873234987258911 Accuracy = 0.9025974273681641
Epoch 1 Step 1200: Loss = 0.19210542738437653 Accuracy = 0.9094244241714478
Epoch 1 Step 1400: Loss = 0.2507779598236084 Accuracy = 0.9146145582199097
Epoch 1 Step 1600: Loss = 0.2379835844039917 Accuracy = 0.9178052544593811
Epoch 1 Step 1800: Loss = 0.17339758574962616 Accuracy = 0.921762228012085
Start of epoch 2
Epoch 2 Step 0: Loss = 0.08006104826927185 Accuracy = 1.0
Epoch 2 Step 200: Loss = 0.17121076583862305 Accuracy = 0.9603545069694519
Epoch 2 Step 400: Loss = 0.13195188343524933 Accuracy = 0.9571384191513062
Epoch 2 Step 600: Loss

## Exercise 3: Custom Callback for Advanced Logging

**Setup Environment**

In [9]:
# Setup the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize the pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0 

# Create a batched dataset for training
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)

**Define Model**

In [10]:
model = Sequential([
    Flatten(input_shape=(28, 28)),  # Flatten the input to a 1D vector
    Dense(128, activation='relu'),  # First hidden layer with 128 neurons and ReLU activation
    Dense(10)  # Output layer with 10 neurons for the 10 classes (digits 0-9)
])

**Define Loss Function, Optimizer, and Metric**

In [11]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Loss function for multi-class classification
optimizer = tf.keras.optimizers.Adam()  # Adam optimizer for efficient training
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()  # Metric to track accuracy during training

**Implement Custom Training Loop with Custom Callback**

In [12]:
class CustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        print(f'End of epoch {epoch + 1}, loss: {logs.get("loss")}, accuracy: {logs.get("accuracy")}')

In [13]:
epochs = 2
custom_callback = CustomCallback()  # Initialize the custom callback

for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')
    
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # Forward pass: Compute predictions
            logits = model(x_batch_train, training=True)
            # Compute loss
            loss_value = loss_fn(y_batch_train, logits)
        
        # Compute gradients
        grads = tape.gradient(loss_value, model.trainable_weights)
        # Apply gradients to update model weights
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        # Update the accuracy metric
        accuracy_metric.update_state(y_batch_train, logits)

        # Log the loss and accuracy every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()} Accuracy = {accuracy_metric.result().numpy()}')
    
    # Call the custom callback at the end of each epoch
    custom_callback.on_epoch_end(epoch, logs={'loss': loss_value.numpy(), 'accuracy': accuracy_metric.result().numpy()})
    
    # Reset the metric at the end of each epoch
    accuracy_metric.reset_state()  # Use reset_state() instead of reset_states()

Start of epoch 1
Epoch 1 Step 0: Loss = 2.2991480827331543 Accuracy = 0.09375
Epoch 1 Step 200: Loss = 0.3921699821949005 Accuracy = 0.8369091749191284
Epoch 1 Step 400: Loss = 0.19065356254577637 Accuracy = 0.8676745891571045
Epoch 1 Step 600: Loss = 0.21626168489456177 Accuracy = 0.8820715546607971
Epoch 1 Step 800: Loss = 0.19728605449199677 Accuracy = 0.8948969841003418
Epoch 1 Step 1000: Loss = 0.3908486068248749 Accuracy = 0.9024413228034973
Epoch 1 Step 1200: Loss = 0.158797025680542 Accuracy = 0.9082275032997131
Epoch 1 Step 1400: Loss = 0.23864497244358063 Accuracy = 0.9131200909614563
Epoch 1 Step 1600: Loss = 0.23799625039100647 Accuracy = 0.9163608551025391
Epoch 1 Step 1800: Loss = 0.21275144815444946 Accuracy = 0.9203914403915405
End of epoch 1, loss: 0.04152049496769905, accuracy: 0.9225500226020813
Start of epoch 2
Epoch 2 Step 0: Loss = 0.07057152688503265 Accuracy = 0.96875
Epoch 2 Step 200: Loss = 0.18653284013271332 Accuracy = 0.9600435495376587
Epoch 2 Step 400: Lo

## Exercise 4: Add Hidden Layers

In [14]:
# Define the input layer
input_layer = Input(shape=(28, 28))  # Input layer with shape (28, 28)

# Define hidden layers
hidden_layer1 = Dense(64, activation='relu')(input_layer)  # First hidden layer with 64 neurons and ReLU activation
hidden_layer2 = Dense(64, activation='relu')(hidden_layer1)  # Second hidden layer with 64 neurons and ReLU activation

## Exercise 5: Define Output Layer

In [15]:
output_layer = Dense(1, activation='sigmoid')(hidden_layer2)

## Exercise 6: Create Model

In [16]:
model = Model(inputs=input_layer, outputs=output_layer)

## Exercise 7: Compile Model

In [17]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## Exercise 8: Train Model

In [18]:
# Redefine the Model for 20 features
model = Sequential([
    Input(shape=(20,)),  # Adjust input shape to (20,)
    Dense(128, activation='relu'),  # Hidden layer with 128 neurons and ReLU activation
    Dense(1, activation='sigmoid')  # Output layer for binary classification with sigmoid activation
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Generate Example Data
X_train = np.random.rand(1000, 20)  # 1000 samples, 20 features each
y_train = np.random.randint(2, size=(1000, 1))  # 1000 binary labels (0 or 1)

# Train the Model
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
32/32 [==============================] - 0s 645us/step - loss: 0.6969 - accuracy: 0.4940
Epoch 2/10
32/32 [==============================] - 0s 646us/step - loss: 0.6901 - accuracy: 0.5400
Epoch 3/10
32/32 [==============================] - 0s 645us/step - loss: 0.6877 - accuracy: 0.5430
Epoch 4/10
32/32 [==============================] - 0s 645us/step - loss: 0.6866 - accuracy: 0.5400
Epoch 5/10
32/32 [==============================] - 0s 645us/step - loss: 0.6840 - accuracy: 0.5620
Epoch 6/10
32/32 [==============================] - 0s 774us/step - loss: 0.6837 - accuracy: 0.5690
Epoch 7/10
32/32 [==============================] - 0s 517us/step - loss: 0.6809 - accuracy: 0.5810
Epoch 8/10
32/32 [==============================] - 0s 645us/step - loss: 0.6790 - accuracy: 0.5750
Epoch 9/10
32/32 [==============================] - 0s 727us/step - loss: 0.6798 - accuracy: 0.5750
Epoch 10/10
32/32 [==============================] - 0s 677us/step - loss: 0.6768 - accuracy: 0.5820

## Exercise 9: Evaluate Model

In [19]:
# Example test data (in practice, use real dataset)
X_test = np.random.rand(200, 20)  # 200 samples, 20 features each
y_test = np.random.randint(2, size=(200, 1))  # 200 binary labels (0 or 1)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test)

# Print test loss and accuracy
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

7/7 [==============================] - 0s 2ms/step - loss: 0.6915 - accuracy: 0.5050
Test loss: 0.6914986371994019
Test accuracy: 0.5049999952316284


## Practice Exercises

## Exercise 1: Basic Custom Training Loop

In [20]:
# Setup the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() 
x_train, x_test = x_train / 255.0, x_test / 255.0 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32) 

# Define the Model
model = Sequential([ 
    Flatten(input_shape=(28, 28)), 
    Dense(128, activation='relu'), 
    Dense(10) 
]) 

# Define Loss Function and Optimizer
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
optimizer = tf.keras.optimizers.Adam() 

# Implement the Custom Training Loop
for epoch in range(5): 
    for x_batch, y_batch in train_dataset: 
        with tf.GradientTape() as tape: 
            logits = model(x_batch, training=True) 
            loss = loss_fn(y_batch, logits) 
        grads = tape.gradient(loss, model.trainable_weights) 
        optimizer.apply_gradients(zip(grads, model.trainable_weights)) 
    print(f'Epoch {epoch + 1}: Loss = {loss.numpy()}')

Epoch 1: Loss = 0.03961790353059769
Epoch 2: Loss = 0.04869624972343445
Epoch 3: Loss = 0.05585509166121483
Epoch 4: Loss = 0.04110407829284668
Epoch 5: Loss = 0.028229668736457825


## Exercise 2: Adding Accuracy Metric

In [21]:
# Setup the Environment
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data() 
x_train = x_train / 255.0 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32) 

# Define the Model
model = Sequential([ 
    Flatten(input_shape=(28, 28)), 
    Dense(128, activation='relu'), 
    Dense(10) 
]) 

# Define Loss Function, Optimizer, and Metric
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
optimizer = tf.keras.optimizers.Adam() 
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy() 

# Implement the Custom Training Loop with Accuracy Tracking
epochs = 5 
for epoch in range(epochs): 
    for x_batch, y_batch in train_dataset: 
        with tf.GradientTape() as tape: 
            logits = model(x_batch, training=True) 
            loss = loss_fn(y_batch, logits) 
        grads = tape.gradient(loss, model.trainable_weights) 
        optimizer.apply_gradients(zip(grads, model.trainable_weights)) 
        accuracy_metric.update_state(y_batch, logits) 
    print(f'Epoch {epoch + 1}: Loss = {loss.numpy()} Accuracy = {accuracy_metric.result().numpy()}') 
    accuracy_metric.reset_state() 

Epoch 1: Loss = 0.03587125241756439 Accuracy = 0.9247000217437744
Epoch 2: Loss = 0.03263261169195175 Accuracy = 0.9649333357810974
Epoch 3: Loss = 0.040505990386009216 Accuracy = 0.9765166640281677
Epoch 4: Loss = 0.02516930177807808 Accuracy = 0.9824833273887634
Epoch 5: Loss = 0.01805526204407215 Accuracy = 0.9873499870300293


## Exercise 3: Custom Callback for Advanced Logging

In [22]:
# Setup the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() 
x_train = x_train / 255.0 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32) 

# Define the Model
model = Sequential([ 
    tf.keras.Input(shape=(28, 28)),  # Updated Input layer syntax
    Flatten(), 
    Dense(128, activation='relu'), 
    Dense(10) 
]) 

# Define Loss Function, Optimizer, and Metric
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
optimizer = tf.keras.optimizers.Adam() 
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy() 

# Implement the Custom Callback
class CustomCallback(Callback): 
    def on_epoch_end(self, epoch, logs=None): 
        print(f'End of epoch {epoch + 1}, loss: {logs.get("loss")}, accuracy: {logs.get("accuracy")}') 

# Implement the Custom Training Loop with Custom Callback
custom_callback = CustomCallback() 

for epoch in range(5): 
    for x_batch, y_batch in train_dataset: 
        with tf.GradientTape() as tape: 
            logits = model(x_batch, training=True) 
            loss = loss_fn(y_batch, logits) 
        grads = tape.gradient(loss, model.trainable_weights) 
        optimizer.apply_gradients(zip(grads, model.trainable_weights)) 
        accuracy_metric.update_state(y_batch, logits) 
    custom_callback.on_epoch_end(epoch, logs={'loss': loss.numpy(), 'accuracy': accuracy_metric.result().numpy()}) 
    accuracy_metric.reset_state()  # Updated method

End of epoch 1, loss: 0.05514167994260788, accuracy: 0.9241833090782166
End of epoch 2, loss: 0.07070286571979523, accuracy: 0.9649999737739563
End of epoch 3, loss: 0.0417470708489418, accuracy: 0.9762499928474426
End of epoch 4, loss: 0.028979355469346046, accuracy: 0.9832333326339722
End of epoch 5, loss: 0.013711516745388508, accuracy: 0.9883166551589966


## Exercise 4: Hyperparameter Tuning

In [23]:
import os
import json

import keras_tuner as kt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Load dataset
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

# Define the model-building function
def build_model(hp):
    model = Sequential()
    # Tune the number of units in the first Dense layer
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32),
                    activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification example
    model.compile(optimizer=Adam(hp.Float('learning_rate', 1e-4, 1e-2, sampling='LOG')),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Initialize a Keras Tuner RandomSearch tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Set the number of trials
    executions_per_trial=1,  # Set how many executions per trial
    directory='tuner_results',  # Directory for saving logs
    project_name='hyperparam_tuning'
)

# Run the tuner search (make sure the data is correct)
tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=5)

# Save the tuning results as JSON files
try:
    for i in range(10):
        # Fetch the best hyperparameters from the tuner
        best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
        
        # Results dictionary to save hyperparameters and score
        results = {
            "trial": i + 1,
            "hyperparameters": best_hps.values,  # Hyperparameters tuned in this trial
            "score": None  # Add any score or metrics if available
        }

        # Save the results as JSON
        with open(os.path.join('tuner_results/hyperparam_tuning/', f"trial_0{i}"), "w") as f:
            json.dump(results, f)

except IndexError:
    print("Tuning process has not completed or no results available.")

Trial 10 Complete [00h 00m 01s]
val_accuracy: 0.9300000071525574

Best val_accuracy So Far: 0.9350000023841858
Total elapsed time: 00h 00m 11s


PermissionError: [Errno 13] Permission denied: 'tuner_results/hyperparam_tuning/trial_00'